<a href="https://colab.research.google.com/github/waleed-o99/Detection-of-Football-Game-Objects/blob/main/Detection_of_Football_Game_Objects_Using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detection of Football Game Objects Using Deep Learning

This project is based on the [YOLOv7](https://github.com/WongKinYiu/yolov7) repository by WongKinYiu.


### **Steps Covered in this Project**

To train our detector we take the following steps:



*   Install YOLOv7 dependencies
*   Login to Wandb
*   Load custom dataset from Roboflow in YOLOv7 format
*   Run YOLOv7 training
*   Evaluate YOLOv7 performance
*   Run YOLOv7 inference on test images
*   OPTIONAL: Deployment




#Install Dependencies

_(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)_

In [ ]:
# Download YOLOv7 repository and install requirements
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

## Install `wandb` library and login

In [ ]:
!pip install wandb
!wandb login

In [ ]:
# start into select project
import wandb
wandb.init(project="test")

# Download Correctly Formatted Custom Data

Next, we'll download our dataset in the right format. Use the `YOLOv7 PyTorch` export. Note that this model requires YOLO TXT annotations, a custom YAML file, and organized directories. The roboflow export writes this for us and saves it in the correct spot.


In [ ]:
!pip install roboflow

In [ ]:
# First dataset project

from roboflow import Roboflow
rf = Roboflow(api_key="ahqVMmM2LD8hcBXgLKtk")
project = rf.workspace("player-detection-ijftp").project("football2-5vahx")
dataset = project.version(4).download("yolov7")

In [ ]:
# Second dataset project

from roboflow import Roboflow
rf = Roboflow(api_key="ahqVMmM2LD8hcBXgLKtk")
project = rf.workspace("player-detection-ijftp").project("football-zqyxh")
dataset = project.version(3).download("yolov7")

Move all Data to one folder

In [ ]:
# Move Test Images & Labels
!mv /content/yolov7/Football2-4/test/images/* /content/yolov7/Football-3/test/images
!mv /content/yolov7/Football2-4/test/labels/* /content/yolov7/Football-3/test/labels

# Move Train Images & Labels
!mv /content/yolov7/Football2-4/train/images/* /content/yolov7/Football-3/train/images
!mv /content/yolov7/Football2-4/train/labels/* /content/yolov7/Football-3/train/labels

# Move Valid Images & Labels
!mv /content/yolov7/Football2-4/valid/images/* /content/yolov7/Football-3/valid/images
!mv /content/yolov7/Football2-4/valid/labels/* /content/yolov7/Football-3/valid/labels

# Begin Custom Training

We're ready to start custom training.

NOTE: We will only modify one of the YOLOv7 training defaults in our example: `epochs`. We will adjust from 300 to 100 epochs in our example for speed. If you'd like to change other settings, see details in [our accompanying blog post](https://blog.roboflow.com/yolov7-custom-dataset-training-tutorial/).

In [ ]:
# download COCO starting checkpoint
%cd /content/yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

In [ ]:
# run this cell to begin training
%cd /content/yolov7
!python train.py --batch 16 --epochs 55 --data {dataset.location}/data.yaml --weights 'yolov7_training.pt' --device 0


In [ ]:
# run this cell to stop Wandb run when the train is complete
import wandb
wandb.finish()


# Evaluation

We can evaluate the performance of our custom training using the provided evalution script.

Note we can adjust the below custom arguments. For details, see [the arguments accepted by detect.py](https://github.com/WongKinYiu/yolov7/blob/main/detect.py#L154).

In [ ]:
# Run evaluation on Image
!python detect.py --weights runs/train/exp/weights/best.pt --conf 0.15 --source {dataset.location}/test/images


In [ ]:
# Run evaluation On video:
python detect.py --weights yolov7.pt --conf 0.25 --img-size 640 --source yourvideo.mp4

In [ ]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

i = 0
limit = 10000 # max images to print
for imageName in glob.glob('/content/yolov7/runs/detect/exp/*.jpg'): #assuming JPG
    if i < limit:
      display(Image(filename=imageName))
      print("\n")
    i = i + 1

# Reparameterize for Inference

https://github.com/WongKinYiu/yolov7/blob/main/tools/reparameterization.ipynb

# OPTIONAL: Deployment

To deploy, you'll need to export your weights and save them to use later.

In [ ]:
# optional, zip to download weights and results locally

!zip -r export1.zip runs/detect
!zip -r export.zip runs/train/exp/weights/best.pt
!zip export.zip runs/train/exp/*

# Referece

*   https://github.com/WongKinYiu/yolov7
*   https://roboflow.com/annotate
*   https://wandb.ai/site